In [12]:
import numpy as np
import pandas as pd
import math
import csv

In [ ]:
#Lê um arquivo de texto chamado "Lista-de-Palavras.txt" e armazena em um objeto do tipo DataFrame utilizando a biblioteca Pandas. 
#O parâmetro header=None indica que não há cabeçalho no arquivo e names=['Palavras'] indica que a única coluna deve ser chamada de "Palavras". 
#O parâmetro dtype=str especifica que o tipo de dado das colunas é string, encoding='utf-8' define a codificação do arquivo e 
#quoting=csv.QUOTE_NONE indica que não deve ser utilizada nenhuma citação para separar os campos. Por fim, na_filter=False desabilita a filtragem de valores nulos.
df = pd.read_csv('Lista-de-Palavras.txt', header=None, names=['Palavras'], dtype=str, encoding='utf-8', quoting=csv.QUOTE_NONE, na_filter=False)

#palavras_banco é uma lista que contém todas as palavras do arquivo "Lista-de-Palavras.txt". 
#palavras_banco_sh é uma lista que contém todas as palavras do arquivo "Lista-de-Palavras.txt" que não contêm o caractere '-'. Isso estava dando um problema.
palavras_banco = df['Palavras'].tolist()
palavras_banco_sh = [palavra for palavra in df['Palavras'] if '-' not in palavra]

#Lista de palavras incorretas que serão corrigidas.
palavras_incorretas = ['BERINJALA', 'ONOMATOPEYA', 'AXUL', 'PARALIZAR', 'ESPOJAR', 'IMPECILHO', 'UVAAZ', 'EXPECIME', 'PROESTANTISMO', 'DESECAR']

In [13]:
#nicialização de listas vazias que armazenarão as correções de cada palavra incorreta.
correcoes_lev = []
correcoes_ang = []
correcoes_euc = []


In [14]:
#Função que calcula a distância de Levenshtein entre duas palavras.
def distancia_levenshtein(palavra1, palavra2):
    m, n = len(palavra1), len(palavra2)
    d = np.zeros((m+1, n+1), dtype=float) # altera o tipo de dado para float
    d[:, 0] = range(m+1)
    d[0, :] = range(n+1)
    for j in range(1, n+1):
        for i in range(1, m+1):
            if palavra1[i-1] == palavra2[j-1]:
                custo = 0
            else:
                custo = 1
            d[i, j] = min(d[i-1, j] + 1,          # deletar
                          d[i, j-1] + 1,          # inserir
                          d[i-1, j-1] + custo)    # substituir
    return d[m, n]

In [15]:
#Função que retorna as cinco menores distâncias e suas respectivas palavras
def obter_cinco_menores(distancias, palavras):
    df = pd.DataFrame({"palavra": palavras, "distancia": distancias})
    df_ordenado = df.sort_values(by="distancia")
    cinco_menores = df_ordenado.head(5)
    return list(zip(cinco_menores["palavra"], cinco_menores["distancia"]))

In [16]:
for palavra_incorreta in palavras_incorretas:
    # Calcula a distância Levenshtein entre a palavra incorreta e cada palavra do banco
    distancias = [distancia_levenshtein(palavra_incorreta, palavra_banco) for palavra_banco in palavras_banco_sh]
    # Obtém as cinco menores distâncias e suas respectivas palavras
    cinco_menores = obter_cinco_menores(distancias, palavras_banco_sh)
    print(f"Palavra incorreta: {palavra_incorreta}")
    for palavra, distancia in cinco_menores:
        print(f"Distância: {distancia}\tPalavra: {palavra}")
    # Adiciona as correções à lista correcoes_lev
    correcoes_lev.extend(cinco_menores)

Palavra incorreta: BERINJALA
Distância: 1.0	Palavra: BERINJELA
Distância: 3.0	Palavra: BENGALA
Distância: 3.0	Palavra: SERINGAL
Distância: 4.0	Palavra: SERENATA
Distância: 4.0	Palavra: GERMINAL
Palavra incorreta: ONOMATOPEYA
Distância: 1.0	Palavra: ONOMATOPEIA
Distância: 3.0	Palavra: ONOMATOPEICO
Distância: 5.0	Palavra: FARMACOPEIA
Distância: 5.0	Palavra: ONOMASTICA
Distância: 6.0	Palavra: BROMATO
Palavra incorreta: AXUL
Distância: 1.0	Palavra: AZUL
Distância: 2.0	Palavra: ATOL
Distância: 2.0	Palavra: SUL
Distância: 2.0	Palavra: UL
Distância: 2.0	Palavra: ATUAL
Palavra incorreta: PARALIZAR
Distância: 1.0	Palavra: PARALISAR
Distância: 2.0	Palavra: BANALIZAR
Distância: 2.0	Palavra: AVALIZAR
Distância: 2.0	Palavra: NASALIZAR
Distância: 2.0	Palavra: PARAFINAR
Palavra incorreta: ESPOJAR
Distância: 1.0	Palavra: ESPOSAR
Distância: 1.0	Palavra: DESPOJAR
Distância: 1.0	Palavra: ESPOCAR
Distância: 2.0	Palavra: ESPORA
Distância: 2.0	Palavra: ESFOLAR
Palavra incorreta: IMPECILHO
Distância: 1.0	Pal